In [98]:
import mongoengine
import meetup.api
import os
import configparser
import requests
import json
import numpy as np
import pandas as pd
from opencage.geocoder import OpenCageGeocode

Configparser implementation

In [19]:
config = configparser.ConfigParser()
config.read('config/meetup.cfg')
os.environ['MEETUP_API_KEY'] = config['MEETUP']['API_KEY']
os.environ['OPENCAGE_KEY']=config['OPENCAGE']['KEY']

geocode = OpenCageGeocode(os.environ['OPENCAGE_KEY'])

In [3]:
class Event:
    
    def __init__(self,
                params,
                url_path):
        self.params = params
        self.url_path = url_path
        
    
    def get_event(self, *args, **kwargs):
        
        r = requests.get(self.url_path, params=self.params)
        request_string = r.text
        
        return request_string

In [103]:
default_args = dict(
    country='United Stated',
    key= os.environ['MEETUP_API_KEY'],
    topic='Python'
)

url_meetup_request = "https://api.meetup.com/2/open_events"
events = Event(params=default_args, url_path=url_meetup_request)
response = events.get_event()

In [104]:
json_response = json.loads(response)
data = json_response['results']
len(data)

200

In [105]:
columns = ['id', 'date', 'year', 'month', 'day', 'country', 'address', 'meetup_name', 'meetup_group_name', 'description', 'event_url', 'yes_rsvp_count', 'status']
id, date, year, month, day, country, address, meetup_name, meetup_group_name, description, event_url, yes_rsvp_count, status = ([] for i in range(13))

for label in data:
    date_event = datetime.fromtimestamp(label['time'] / 1000.0)
    location_json = geocode.reverse_geocode(label['group']['group_lat'], label['group']['group_lon'])
    location_meetup = location_json[0]['components']
    address_meetup = location_json[0]['formatted']
    
    
    id.append(label['id'])
    date.append(date_event)
    year.append(date_event.year)
    month.append(date_event.month)
    day.append(date_event.year)
    country.append(location_meetup['country'])
    address.append(address_meetup)
    meetup_name.append(label['name'])
    meetup_group_name.append(label['group']['name'])
    description.append(label['description'])
    event_url.append(label['event_url'])
    yes_rsvp_count.append(label['yes_rsvp_count'])
    status.append(label['status'])

In [112]:
# results_geocode = geocode.reverse_geocode(label['group']['group_lat'], label['group']['group_lon'])

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.5") \
    .getOrCreate()

schema = T.StructType([
        T.StructField("id", T.StringType(), True),
        T.StructField("date", T.TimestampType(), True),
        T.StructField("year", T.IntegerType(), True),
        T.StructField("month", T.IntegerType(), True),
        T.StructField("day", T.IntegerType(), True),
        T.StructField("country", T.StringType(), True),
        T.StructField("address", T.StringType(), True),
        T.StructField("meetup_name", T.StringType(), True),
        T.StructField("meetup_group_name", T.StringType(), True),
        T.StructField("description", T.StringType(), True),
        T.StructField("event_url", T.StringType(), True),
        T.StructField("status", T.StringType(), True)  
])

df_pandas = pd.DataFrame(np.transpose([id, date, year, month, day, country, address, meetup_name, meetup_group_name, description, event_url, yes_rsvp_count, status]), columns=columns)
df = spark.createDataFrame(df_pandas, schema=schema)
df.limit(5).toPandas()

ValueError: Length of object (13) does not match with length of fields (12)

,id,date,year,month,day,country,address,meetup_name,meetup_group_name,description,event_url,yes_rsvp_count,status
0,257904670,2019-05-08 08:30:00,2019,5,2019,USA,"333 East 43rd Street, New York City, NY 10017,...",Finovate Spring,"Byte Academy: Python, FinTech, Blockchain,Data...",<p>We're thrilled to offer meetup members a sp...,https://www.meetup.com/Byte-Academy-Finance-an...,5,upcoming
1,259042717,2019-05-08 03:00:00,2019,5,2019,Germany,"B 80, 06108 Halle (Saale), Germany",Saltlabs sprint - Zope spring cleaning,Zope Sprints,<p>As we enter the last year of Python 2 suppo...,https://www.meetup.com/Zope-Sprint/events/2590...,10,upcoming
2,trjvpqyzhblb,2019-05-08 20:30:00,2019,5,2019,USA,"Northwest Lovejoy Street, Portland, OR 97217, ...",JavaScript Study Night,Women Who Code Portland,<p>May JavaScript Study Night - TypeScript wit...,https://www.meetup.com/Women-Who-Code-Portland...,43,upcoming
3,261022873,2019-05-08 19:30:00,2019,5,2019,USA,"9389 East Orchard Road, Greenwood Village, CO ...",Kuber Party,DevOps Exchange DTC Meetup,<p>Join us for an informal get together with r...,https://www.meetup.com/DevOps-Exchange-DTC-Mee...,6,upcoming
4,261037105,2019-05-08 20:30:00,2019,5,2019,USA,"University of California, San Diego, 9500 Gilm...",Jupyter Notebook in Science and Education at UCSD,San Diego Jupyter Meetup,<p>Location: SDSC Auditorium</p> <p>Agenda:<br...,https://www.meetup.com/San-Diego-Jupyter-Meetu...,18,upcoming
